In [1]:
#IMPORTING MODULES
import pandas as pd
import numpy as np
from tkinter import *
from tkinter import filedialog, messagebox
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#SETTING GLOBAL VARIABLES
dataset = None
X_train, X_test, Y_train, Y_test = None, None, None, None
classifier = None

#FUNCTION FOR UPLOADING DATASET
def upload_dataset():
    global dataset
    filepath = filedialog.askopenfilename(title="Upload Dataset", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if filepath:
        dataset = pd.read_csv(filepath)
        messagebox.showinfo("Success", "Dataset uploaded successfully.")
    else:
        messagebox.showerror("Error", "Dataset upload failed.")

#FUNCTION FOR STARTING CLASSIFICATION
def start_classification():
    global dataset, X_train, X_test, Y_train, Y_test, classifier
    if dataset is None:
        messagebox.showerror("Error", "Please upload a dataset first.")
        return

    X = dataset.iloc[:, 2:].values
    Y = dataset.iloc[:, 1].values

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, Y_train)

    messagebox.showinfo("Success", "Classification started successfully.")

#FUNCTION TO FIND ACCURACY OF THE CLASSIFICATION SYSTEM
def find_accuracy():
    global X_train, X_test, Y_train, Y_test, classifier
    if dataset is None:
        messagebox.showerror("Error", "Please upload a dataset first.")
        return
    if X_train is None or X_test is None or Y_train is None or Y_test is None or classifier is None:
        messagebox.showerror("Error", "Please start the classification first.")
        return

    Y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    messagebox.showinfo("Accuracy", "Accuracy Score: {:.2f}%".format(accuracy * 100))

#FUNCTION FOR ENTERING USER INPUT VALUES   
def enter_user_data():
    global classifier
    if dataset is None:
        messagebox.showerror("Error", "Please upload a dataset first.")
        return
    if classifier is None:
        messagebox.showerror("Error", "Please start the classification first.")
        return

    window = Toplevel()
    window.title("Enter User Data")
    window.configure(bg='light blue')

    label = Label(window, text="Enter values for the following columns:")
    label.pack()

    user_data = {}

    for column in dataset.columns[2:]:
        frame = Frame(window)
        frame.pack(pady=2)

        label = Label(frame, text=column)
        label.pack(side=LEFT)

        entry = Entry(frame, width=10)
        entry.pack(side=RIGHT)

        user_data[column] = entry

    button = Button(window, text="Classify", command=lambda: classify_user_data(user_data))
    button.pack()

#FUNCTION FOR FINDING THE CLASSIFICATION FOR ENTERED VALUE SET
def classify_user_data(user_data):
    global classifier
    if dataset is None:
        messagebox.showerror("Error", "Please upload a dataset first.")
        return
    if classifier is None:
        messagebox.showerror("Error", "Please start the classification first.")
        return

    input_data = []
    for column in dataset.columns[2:]:
        input_data.append(float(user_data[column].get()))

    input_data = np.array(input_data).reshape(1, -1)  # Convert to NumPy array

    # Map class labels to numeric values
    class_mapping = {'B': 0, 'M': 1}
    predicted_class = classifier.predict(input_data)[0]
    prediction = class_mapping[predicted_class]
    probability = classifier.predict_proba(input_data)[0][prediction]

    classification = "Benign" if prediction == 0 else "Malignant"
    messagebox.showinfo("Classification", "The user might have {} classification of breast cancer with a probability of {:.2f}%".format(classification, probability * 100))

#UI WINDOW
window = Tk()
window.title("Breast Cancer Classification")
window.geometry("400x200")
window.configure(bg='pink')

#BUTTONS
upload_button = Button(window, text="Upload Dataset", command=upload_dataset)
upload_button.pack(pady=10)

classification_button = Button(window, text="Start Classification", command=start_classification)
classification_button.pack(pady=10)

accuracy_button = Button(window, text="Find Accuracy", command=find_accuracy)
accuracy_button.pack(pady=10)

user_data_button = Button(window, text="Enter User Data", command=enter_user_data)
user_data_button.pack(pady=10)

#INITIALIZING THE APPLICATION
window.mainloop()